In [32]:
#!pip install python-dotenv
#!pip install pyautogen

In [25]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
llm_config = {"model": "gpt-3.5-turbo"}

In [27]:
from autogen import ConversableAgent

In [28]:
agent = ConversableAgent(
    name="chatbot",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [30]:
reply = agent.generate_reply(
    messages=[{"content": "Tell me a joke.", "role": "user"}]
)
print(reply)

Sure, here's one for you:

Why don't scientists trust atoms?

Because they make up everything!


In [31]:
reply = agent.generate_reply(
    messages=[{"content": "Repeat the joke.", "role": "user"}]
)
print(reply)

Of course! Just let me know which joke you'd like me to repeat for you.


## Conversation

In [33]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "Start the next joke from the punchline of the previous joke.",
    llm_config=llm_config,
    human_input_mode="NEVER",
)

In [34]:
chat_result = joe.initiate_chat(
    recipient=cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.",
    max_turns=2,
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Of course, Joe! So, have you ever noticed that parents always tell you not to play with your food, but then they turn around and make smiley faces out of your veggies? Like, which one is it, Mom and Dad? Am I Picasso or a hungry child?

--------------------------------------------------------------------------------
joe (to cathy):

It's like they want you to be a starving artist!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly! They're trying to create the next masterpiece while also ensuring you don't actually eat it. It's a culinary conundrum, Joe!

--------------------------------------------------------------------------------


## Print some results

In [35]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "I'm Joe. Cathy, let's keep the jokes rolling.",
  'role': 'assistant'},
 {'content': 'Of course, Joe! So, have you ever noticed that parents always '
             'tell you not to play with your food, but then they turn around '
             'and make smiley faces out of your veggies? Like, which one is '
             'it, Mom and Dad? Am I Picasso or a hungry child?',
  'role': 'user'},
 {'content': "It's like they want you to be a starving artist!",
  'role': 'assistant'},
 {'content': "Exactly! They're trying to create the next masterpiece while "
             "also ensuring you don't actually eat it. It's a culinary "
             'conundrum, Joe!',
  'role': 'user'}]


In [36]:
pprint.pprint(chat_result.cost)

{'usage_excluding_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 99,
                                                             'cost': 0.00027800000000000004,
                                                             'prompt_tokens': 259,
                                                             'total_tokens': 358},
                                      'total_cost': 0.00027800000000000004},
 'usage_including_cached_inference': {'gpt-3.5-turbo-0125': {'completion_tokens': 99,
                                                             'cost': 0.00027800000000000004,
                                                             'prompt_tokens': 259,
                                                             'total_tokens': 358},
                                      'total_cost': 0.00027800000000000004}}


In [37]:
pprint.pprint(chat_result.summary)

("Exactly! They're trying to create the next masterpiece while also ensuring "
 "you don't actually eat it. It's a culinary conundrum, Joe!")


## Get a better summary of the conversation

In [38]:
chat_result = joe.initiate_chat(
    cathy, 
    message="I'm Joe. Cathy, let's keep the jokes rolling.", 
    max_turns=2, 
    summary_method="reflection_with_llm",
    summary_prompt="Summarize the conversation",
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Of course, Joe! So, have you ever noticed that parents always tell you not to play with your food, but then they turn around and make smiley faces out of your veggies? Like, which one is it, Mom and Dad? Am I Picasso or a hungry child?

--------------------------------------------------------------------------------
joe (to cathy):

It's like they want you to be a starving artist!

--------------------------------------------------------------------------------
cathy (to joe):

Exactly! They're trying to create the next masterpiece while also ensuring you don't actually eat it. It's a culinary conundrum, Joe!

--------------------------------------------------------------------------------


In [39]:
pprint.pprint(chat_result.summary)

('Parents tell kids not to play with their food but encourage creativity by '
 'making smiley faces out of it, creating a culinary conundrum.')


## Chat termination

In [40]:
cathy = ConversableAgent(
    name="cathy",
    system_message=
    "Your name is Cathy and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"],
)

joe = ConversableAgent(
    name="joe",
    system_message=
    "Your name is Joe and you are a stand-up comedian. "
    "When you're ready to end the conversation, say 'I gotta go'.",
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"] or "Goodbye" in msg["content"],
)

In [41]:
chat_result = joe.initiate_chat(
    recipient=cathy,
    message="I'm Joe. Cathy, let's keep the jokes rolling."
)

joe (to cathy):

I'm Joe. Cathy, let's keep the jokes rolling.

--------------------------------------------------------------------------------
cathy (to joe):

Hey Joe, nice to meet you! Why did the scarecrow win an award? Because he was outstanding in his field!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Here's another one for you: Why couldn't the bicycle stand up by itself? Because it was two tired!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, good one, Joe! How about this one: Why did the math book look sad? Because it had too many problems.

--------------------------------------------------------------------------------
joe (to cathy):

Haha, I love it! Math books always have a lot of problems, don't they? Here's one more for you: Why did the tomato turn red? Because it saw the salad dressing!

-----------------------------------

In [42]:
cathy.send(message="What's last joke we talked about?", recipient=joe)

cathy (to joe):

What's last joke we talked about?

--------------------------------------------------------------------------------
joe (to cathy):

The last joke we talked about was, Why did the coffee file a police report? It got mugged! That was a good one!

--------------------------------------------------------------------------------
cathy (to joe):

That's right! Thanks for remembering, Joe! Keep spreading the laughter and have a fantastic day! I gotta go.

--------------------------------------------------------------------------------
